# NetCDF and xarray

In [4]:
import os
import urllib
import pandas as pd
import numpy as np
import xarray as xr

## Creating Xarray

In [ ]:
temp_data = np.array([np.zeros((5,5)),
                      np.ones((5,5)),
                    2*np.ones((5,5))]).astype(int)

print(temp_data)

In [9]:
dims = ('time','lat','lon')
coords = {'time':pd.date_range("2022-09-01","2022-09-03"),
         'lat': np.arange(70,20,-10), # from 70 to 10 with step -10
         'lon': np.arange(60,110,10)}

In [6]:
attrs = {'title':'example temperature data',
         'standard name': 'air_temperature',
         'units': 'degree_c'}

In [12]:
temp = xr.DataArray(data   = temp_data,
                    dims   = dims,
                    coords = coords,
                    attrs  = attrs)
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)> Size: 600B
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:          example temperature data
    standard name:  air_temperature
    units:          degree_c

## Updating attributes

In [15]:
# update attributes
temp.attrs['description'] = 'simple example of an xarray.DataArray'

# add attributes to coordinates 
temp.time.attrs = {'description':'date of measurement'}

temp.lat.attrs['standard_name']= 'grid_latitude'
temp.lat.attrs['units'] = 'degree_N'

temp.lon.attrs['standard_name']= 'grid_longitude'
temp.lon.attrs['units'] = 'degree_E'
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)> Size: 600B
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:          example temperature data
    standard name:  air_temperature
    units:          degree_c
    description:    simple example of an xarray.DataArray

## Accesing using indexing

In [17]:
# select tempeature data that at given attributes
temp.sel(time = "2022-09-01", lat = 40, lon = 80, method='nearest')

<xarray.DataArray ()> Size: 8B
array(0)
Coordinates:
    time     datetime64[ns] 8B 2022-09-01
    lat      int64 8B 40
    lon      int64 8B 80
Attributes:
    title:          example temperature data
    standard name:  air_temperature
    units:          degree_c
    description:    simple example of an xarray.DataArray

In [18]:
# select tempeature data that at some lat which is not on grided point
temp.sel(time = "2022-09-01", lat = 44, lon = 80, method='nearest')

<xarray.DataArray ()> Size: 8B
array(0)
Coordinates:
    time     datetime64[ns] 8B 2022-09-01
    lat      int64 8B 40
    lon      int64 8B 80
Attributes:
    title:          example temperature data
    standard name:  air_temperature
    units:          degree_c
    description:    simple example of an xarray.DataArray

In [19]:
# select tempeature data that at given range of lat
temp.sel(time = "2022-09-01", lat = np.arange(40,60,10), lon = 80)

<xarray.DataArray (lat: 2)> Size: 16B
array([0, 0])
Coordinates:
    time     datetime64[ns] 8B 2022-09-01
  * lat      (lat) int64 16B 40 50
    lon      int64 8B 80
Attributes:
    title:          example temperature data
    standard name:  air_temperature
    units:          degree_c
    description:    simple example of an xarray.DataArray

In [37]:
# select tempeature data that at given range of date range
temp.sel(time = pd.date_range("2022-09-01","2022-09-02"), lat = 40, lon = 80)

<xarray.DataArray (time: 2)> Size: 16B
array([0, 1])
Coordinates:
  * time     (time) datetime64[ns] 16B 2022-09-01 2022-09-02
    lat      int64 8B 40
    lon      int64 8B 80
Attributes:
    title:          example temperature data
    standard name:  air_temperature
    units:          degree_c
    description:    simple example of an xarray.DataArray

## Reduction method

In [21]:
# taking average over the time dimension, after this, the third dim is gone
avg_temp = temp.mean(dim = 'time')
avg_temp.attrs['title'] = 'average temperature over 3 days'
avg_temp

<xarray.DataArray (lat: 5, lon: 5)> Size: 200B
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:    average temperature over 3 days

## Create xarray.DataSet using xarray

In [22]:
data_vars = {'avg_temp': avg_temp,
             'temp':     temp}

attrs ={'title': 'averaged and daily temperature data',
        'description': 'example xarray.DataSet'}

temp_dataset = xr.Dataset(data_vars = data_vars,
                          attrs = attrs)

temp_dataset

<xarray.Dataset> Size: 904B
Dimensions:   (lat: 5, lon: 5, time: 3)
Coordinates:
  * lat       (lat) int64 40B 70 60 50 40 30
  * lon       (lon) int64 40B 60 70 80 90 100
  * time      (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
Data variables:
    avg_temp  (lat, lon) float64 200B 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    temp      (time, lat, lon) int64 600B 0 0 0 0 0 0 0 0 0 ... 2 2 2 2 2 2 2 2
Attributes:
    title:        averaged and daily temperature data
    description:  example xarray.DataSet

## Read and write xarray.DataSet to netCDF

In [23]:
fp = os.path.join(os.getcwd(), 'temp_dataset.nc')
temp_dataset.to_netcdf(fp)

check = xr.open_dataset(fp)
check

<xarray.Dataset> Size: 904B
Dimensions:   (lat: 5, lon: 5, time: 3)
Coordinates:
  * lat       (lat) int64 40B 70 60 50 40 30
  * lon       (lon) int64 40B 60 70 80 90 100
  * time      (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
Data variables:
    avg_temp  (lat, lon) float64 200B ...
    temp      (time, lat, lon) int64 600B ...
Attributes:
    title:        averaged and daily temperature data
    description:  example xarray.DataSet

## Read in a real ADC dataset

In [24]:
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3A792bfc37-416e-409e-80b1-fdef8ab60033'

msg = urllib.request.urlretrieve(url, "FW_data_CESM_LW_2006_2100.nc")

In [25]:
fp = os.path.join(os.getcwd(), "FW_data_CESM_LW_2006_2100.nc")
fw_data = xr.open_dataset(fp)
fw_data

<xarray.Dataset> Size: 135kB
Dimensions:                          (time: 95, member: 11)
Coordinates:
  * time                             (time) float64 760B 2.006e+03 ... 2.1e+03
  * member                           (member) float64 88B 1.0 2.0 ... 10.0 11.0
Data variables: (12/16)
    FW_flux_Fram_annual_net          (time, member) float64 8kB ...
    FW_flux_Barrow_annual_net        (time, member) float64 8kB ...
    FW_flux_Nares_annual_net         (time, member) float64 8kB ...
    FW_flux_Davis_annual_net         (time, member) float64 8kB ...
    FW_flux_BSO_annual_net           (time, member) float64 8kB ...
    FW_flux_Bering_annual_net        (time, member) float64 8kB ...
    ...                               ...
    Solid_FW_flux_BSO_annual_net     (time, member) float64 8kB ...
    Solid_FW_flux_Bering_annual_net  (time, member) float64 8kB ...
    runoff_annual                    (time, member) float64 8kB ...
    netPrec_annual                   (time, member) float64 8kB ...
    Liquid_FW_storage_Arctic_annual  (time, member) float64 8kB ...
    Solid_FW_storage_Arctic_annual   (time, member) float64 8kB ...
Attributes:
    creation_date:   02-Jun-2020 15:38:31
    author:          Alexandra Jahn, CU Boulder, alexandra.jahn@colorado.edu
    title:           Annual timeseries of freshwater data from the CESM Low W...
    description:     Annual mean Freshwater (FW) fluxes and storage relative ...
    data_structure:  The data structure is |Ensemble member | Time (in years)...

## Select specific data array from xarray.DataSet

In [27]:
# select values for second ensemble member of netPrec_annual variable
member2 = fw_data.netPrec_annual.sel(member = 2)
member2

<xarray.DataArray 'netPrec_annual' (time: 95)> Size: 760B
[95 values with dtype=float64]
Coordinates:
  * time     (time) float64 760B 2.006e+03 2.007e+03 ... 2.099e+03 2.1e+03
    member   float64 8B 2.0
Attributes:
    units:      km3/yr
    long_name:  Net FW flux from precipitation minus evaporation over the Arc...

In [39]:
# this only returns the max value
xmax = member2.sel(time = np.arange(2022,2101)).max()
xmax.item() # return the single data and dump all descriptions

# if want to retain the date. i.e. index of the data
xmax = member2.sel(time = np.arange(2022,2101)).max(keepdims = True)
xmax

<xarray.DataArray 'netPrec_annual' (time: 1)> Size: 8B
array([2431.10032351])
Coordinates:
    member   float64 8B 2.0
Dimensions without coordinates: time